In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models import resnet18
import numpy as np
import shap
import cv2
import json
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp

In [20]:
# --------------------------
# Step 1: Configuration
# --------------------------
class Config:
    IMAGE_DIR = "./BDDD100K/train/images"
    LABEL_FILE = "./BDDD100K/train/annotations/bdd100k_labels_images_train.json"
    SEG_LABEL_DIR = "bdd100k/labels/segmentation"
    NUM_CLASSES = 9  # [brake, steer_left, steer_right, accelerate, lane_change_left, lane_change_right, maintain_lane, stop_completely, overtake]
    INPUT_SIZE = (224, 224)
    BATCH_SIZE = 32
    EPOCHS = 0
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    @classmethod
    def print_cuda_info(cls):
        print("\nCUDA Information:")
        print(f"CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"Current device: {torch.cuda.current_device()}")
            print(f"Device name: {torch.cuda.get_device_name()}")
            print(f"Device count: {torch.cuda.device_count()}")
            print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
            print(f"Memory cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")

config = Config()


In [21]:
# --------------------------
# Step 2: Dataset Preparation
# --------------------------
class BDD100KHMI(Dataset):
    def __init__(self, split='train', transform=None):
        # Set the appropriate paths based on split
        if split == 'train':
            self.image_dir = config.IMAGE_DIR
            label_file = config.LABEL_FILE
        elif split == 'val':
            self.image_dir = config.IMAGE_DIR
            label_file = config.LABEL_FILE
        elif split == 'test':
            self.image_dir = config.IMAGE_DIR
            self.data = []  # No labels for test set
            return
        
        # Load annotations if not test set
        with open(label_file, 'r') as f:
            self.data = json.load(f)
            
        self.transform = transform or T.Compose([
            T.ToPILImage(),
            T.Resize(config.INPUT_SIZE),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        img_path = f"{self.image_dir}/{entry['name']}"
        
        # Load image
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image)

        # For test set, return only the image
        if not hasattr(self, 'data') or not self.data:
            return image

        # Generate labels based on annotations
        label = 6  # Default: Maintain Lane
        for obj in entry.get('labels', []):
            if obj['category'] == 'pedestrian':
                label = 0  # Brake
            elif obj['category'] == 'car':
                label = 1  # Steer Left
            elif obj['category'] == 'traffic light' and obj.get('attributes', {}).get('trafficLightColor') == 'red':
                label = 2  # Steer Right
            elif obj['category'] == 'bicycle':
                label = 3  # Accelerate
            elif obj['category'] == 'lane_marking' and obj.get('attributes', {}).get('change') == 'left':
                label = 4  # Lane Change Left
            elif obj['category'] == 'lane_marking' and obj.get('attributes', {}).get('change') == 'right':
                label = 5  # Lane Change Right
            elif obj['category'] == 'stop_sign':
                label = 7  # Stop Completely
            elif obj['category'] == 'slow_vehicle':
                label = 8  # Overtake

        return image, label



In [22]:
# --------------------------
# Step 3: Model Definition
# --------------------------
def build_models():
    # Object Detection Model (YOLOv5)
    obj_detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Segmentation Model (U-Net)
    seg_model = smp.Unet(encoder_name="resnet18", encoder_weights="imagenet", in_channels=3, classes=1)

    # Decision Model
    decision_model = resnet18(pretrained=True)
    decision_model.fc = nn.Linear(decision_model.fc.in_features, config.NUM_CLASSES)

    return obj_detector, seg_model.to(config.DEVICE), decision_model.to(config.DEVICE)


In [23]:
# --------------------------
# Step 4: Training
# --------------------------
def train_model(decision_model, dataloader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(decision_model.parameters(), lr=0.001)
    
    # Add tracking variables
    best_loss = float('inf')
    total_batches = len(dataloader)
    
    print(f"\nStarting training on {config.DEVICE}")
    print(f"Total batches per epoch: {total_batches}")
    
    # Add this check
    if config.DEVICE == "cuda":
        print(f"Training on GPU: {torch.cuda.get_device_name()}")
        decision_model = decision_model.cuda()
    
    decision_model.train()
    for epoch in range(config.EPOCHS):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        # Add progress bar with tqdm
        progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{config.EPOCHS}')
        
        for batch_idx, (images, labels) in enumerate(progress_bar):
            # Explicitly move to GPU with non_blocking=True for better performance
            images = images.to(config.DEVICE, non_blocking=True)
            labels = labels.to(config.DEVICE, non_blocking=True)
            
            optimizer.zero_grad()
            outputs = decision_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            running_loss += loss.item()
            
            # Add memory tracking in progress bar
            if config.DEVICE == "cuda":
                gpu_memory = torch.cuda.memory_allocated() / 1024**2
                progress_bar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'avg_loss': f'{running_loss/(batch_idx+1):.4f}',
                    'accuracy': f'{100*correct_predictions/total_samples:.2f}%',
                    'GPU Memory': f'{gpu_memory:.1f}MB'
                })
        
        # Epoch summary
        epoch_loss = running_loss/total_batches
        epoch_accuracy = 100 * correct_predictions/total_samples
        print(f"\nEpoch [{epoch+1}/{config.EPOCHS}] Summary:")
        print(f"Average Loss: {epoch_loss:.4f}")
        print(f"Accuracy: {epoch_accuracy:.2f}%")
        
        # Save best model
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            print(f"New best loss achieved! Saving model checkpoint...")
            torch.save(decision_model.state_dict(), 'best_model.pth')


In [24]:
# --------------------------
# Step 5: Explainability with SHAP
# --------------------------
def explain_with_shap(decision_model, dataset):
    print("\nGenerating SHAP explanations...")
    decision_model.eval()
    
    print("Preparing background samples...")
    background = torch.stack([dataset[i][0] for i in range(100)]).to(config.DEVICE)
    
    print("Creating SHAP explainer...")
    explainer = shap.DeepExplainer(decision_model, background)

    print("Generating explanations for test images...")
    test_images = torch.stack([dataset[i][0] for i in range(5)]).to(config.DEVICE)
    shap_values = explainer.shap_values(test_images)

    print("Visualizing SHAP values...")
    for i in range(5):
        plt.figure(figsize=(5, 5))
        shap.image_plot(shap_values, np.transpose(test_images.cpu().numpy(), (0, 2, 3, 1)))
        plt.show()



In [25]:
# --------------------------
# Step 6: Main Pipeline
# --------------------------
if __name__ == "__main__":
    print("\n=== Starting BDD100K HMI Model Pipeline ===\n")
    
    # Add this right at the start
    Config.print_cuda_info()
    
    # Force CUDA memory clearance if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    print("1. Initializing Datasets...")
    train_dataset = BDD100KHMI(split='train')
    print(f"   - Train dataset size: {len(train_dataset)} samples")
    val_dataset = BDD100KHMI(split='val')
    print(f"   - Validation dataset size: {len(val_dataset)} samples")
    print("✓ Datasets initialized successfully\n")
    
    print("2. Creating DataLoaders...")
    train_dataloader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    print(f"   - Train batches: {len(train_dataloader)}")
    val_dataloader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)
    print(f"   - Validation batches: {len(val_dataloader)}")
    print("✓ DataLoaders created successfully\n")

    print("3. Building Models...")
    print("   - Loading YOLOv5...")
    obj_detector, seg_model, decision_model = build_models()
    print("   - Loading Segmentation Model...")
    print("   - Loading Decision Model...")
    print(f"✓ All models loaded successfully (using {config.DEVICE})\n")

    print("4. Starting Model Training...")
    train_model(decision_model, train_dataloader)
    print("✓ Training completed\n")

    print("5. Generating SHAP Explanations...")
    explain_with_shap(decision_model, train_dataset)
    print("✓ SHAP analysis completed\n")

    print("6. Collecting Decision Labels...")
    decision_labels = train_dataset.get_all_labels()
    print("   Decision Labels Distribution:")
    unique_labels, counts = np.unique(decision_labels, return_counts=True)
    for label, count in zip(unique_labels, counts):
        print(f"   - Class {label}: {count} samples")
    print("✓ Label collection completed\n")

    print("=== Pipeline Completed Successfully ===")



=== Starting BDD100K HMI Model Pipeline ===


CUDA Information:
CUDA available: True
Current device: 0
Device name: NVIDIA GeForce GTX 1650
Device count: 1
Memory allocated: 11.44 MB
Memory cached: 22.00 MB
1. Initializing Datasets...
   - Train dataset size: 69863 samples
   - Validation dataset size: 69863 samples
✓ Datasets initialized successfully

2. Creating DataLoaders...
   - Train batches: 2184
   - Validation batches: 2184
✓ DataLoaders created successfully

3. Building Models...
   - Loading YOLOv5...


Using cache found in C:\Users\VICTUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-2 Python-3.12.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


   - Loading Segmentation Model...
   - Loading Decision Model...
✓ All models loaded successfully (using cuda)

4. Starting Model Training...

Starting training on cuda
Total batches per epoch: 2184
Training on GPU: NVIDIA GeForce GTX 1650


Epoch 1/1: 100%|█| 2184/2184 [28:01<00:00,  1.30it/s, loss=0.1134, avg_loss=0.3239, accuracy=90.71%, 



Epoch [1/1] Summary:
Average Loss: 0.3239
Accuracy: 90.71%
New best loss achieved! Saving model checkpoint...
✓ Training completed

5. Generating SHAP Explanations...

Generating SHAP explanations...
Preparing background samples...
Creating SHAP explainer...
Generating explanations for test images...


RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.

In [ ]:
# loading best mdoel,

In [16]:
import torch
import shap
import numpy as np
import matplotlib.pyplot as plt

def explain_with_shap(decision_model, dataset):
    print("\nGenerating SHAP explanations...")
    decision_model.eval()

    # Optimize memory usage
    torch.cuda.empty_cache()  # Clear cached memory

    print("Preparing background samples...")
    background_size = 4  # Reduced for GTX 1650
    background = torch.stack([dataset[i][0] for i in range(background_size)]).to(config.DEVICE)

    print("Creating SHAP explainer...")
    explainer = shap.DeepExplainer(decision_model, background)

    print("Generating explanations for test images in batches...")
    batch_size = 2  # Process smaller batches
    num_test_images = 5  # Total images to explain

    for i in range(0, num_test_images, batch_size):
        torch.cuda.empty_cache()  # Clear cache between batches

        batch_images = torch.stack([
            dataset[j][0] for j in range(i, min(i + batch_size, num_test_images))
        ]).to(config.DEVICE)

        with torch.cuda.amp.autocast():  # Optional: Mixed Precision
            shap_values = explainer.shap_values(batch_images)

        print(f"Visualizing SHAP values for batch {i // batch_size + 1}...")
        for idx in range(batch_images.size(0)):
            plt.figure(figsize=(5, 5))
            shap.image_plot(
                shap_values, 
                np.transpose(batch_images.cpu().numpy(), (0, 2, 3, 1))
            )
            plt.show()

        del batch_images, shap_values  # Free memory

    torch.cuda.empty_cache()  # Final cleanup
    print("✓ SHAP analysis completed\n")


In [17]:
# Function to load the pre-trained model
def load_model(model, path):
    print(f"Loading model from {path}...")
    checkpoint = torch.load(path, map_location=config.DEVICE)
    model.load_state_dict(checkpoint)
    model.to(config.DEVICE)
    print("✓ Model loaded successfully\n")
    return model


In [18]:
# ----- Load Pre-trained Model -----
model_path = "./best_model.pth"  # Specify your .pth file here
decision_model = load_model(decision_model, model_path)
torch.cuda.empty_cache()  # Clear GPU memory after loading
# ----------------------------------

# Skip training since the model is pre-trained
print("4. Skipping Model Training (pre-trained model loaded)\n")

print("5. Generating SHAP Explanations...")
explain_with_shap(decision_model, train_dataset)
print("✓ SHAP analysis completed\n")

print("6. Collecting Decision Labels...")
decision_labels = train_dataset.get_all_labels()
print("   Decision Labels Distribution:")
unique_labels, counts = np.unique(decision_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"   - Class {label}: {count} samples")
print("✓ Label collection completed\n")

print("=== Pipeline Completed Successfully ===")

NameError: name 'decision_model' is not defined

In [ ]:
print(torch.__version__)          # PyTorch version
print(torch.version.cuda)         # CUDA version PyTorch was built with


In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0))

# Simple GPU test
x = torch.randn(1000, 1000).to('cuda')
y = torch.randn(1000, 1000).to('cuda')
z = x + y
print("Tensor on GPU:", z.device)


CUDA Available: True
Device Name: NVIDIA GeForce GTX 1650
Tensor on GPU: cuda:0


In [2]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0))

# Smaller tensor test
x = torch.randn(10, 10).to('cuda')
y = torch.randn(10, 10).to('cuda')
z = x + y
print("Tensor on GPU:", z.device)


CUDA Available: True
Device Name: NVIDIA GeForce GTX 1650
Tensor on GPU: cuda:0


In [22]:
pip install pycuda


^C
Note: you may need to restart the kernel to use updated packages.
